In [1]:
# import
import os
from glob import glob
import pandas as pd
import duckdb

In [2]:
# read SCCF (by region)
sheets = ['Sylhet', 'Khulna', 'Bogra', 'CN', 'CS', 'CTG']
file = r"C:\Users\Shithi.Maitra\Downloads\Outlet-Channel Wise Sec CCF JAN-JULY 2023.xlsx"
sccf_df = pd.DataFrame()
for s in sheets:
    print("Reading data from: " + s)
    df = pd.read_excel(open(file, "rb"), sheet_name=s, header=1, index_col=None)
    df['region'] = s
    sccf_df = sccf_df.append(df)
    
# clean SCCF
sccf_df = sccf_df.applymap(lambda s: s.upper() if type(s)==str else s)
sccf_df.columns = ['month', 'town', 'outlet', 'channel', 'ord_qty', 'inv_qty', 'loss_qty', 'ccf', 'region']
display(sccf_df)

Reading data from: Sylhet
Reading data from: Khulna
Reading data from: Bogra
Reading data from: CN
Reading data from: CS
Reading data from: CTG


,month,town,outlet,channel,ord_qty,inv_qty,loss_qty,ccf,region
0,1,ARAIHAZAR,C16-0092,WHOLESALE,43.570,43.226,0.344,99.211039,SYLHET
1,1,ARAIHAZAR,C16-0113,RURAL NEIGHBORHOOD GROCER,1.612,1.500,0.111,93.105147,SYLHET
2,1,ARAIHAZAR,C16-0142,RURAL WET MARKET GROCER,1.155,1.121,0.035,96.994332,SYLHET
3,1,ARAIHAZAR,C16-0165,URBAN WET MARKET GROCER,1.617,1.554,0.063,96.134574,SYLHET
4,1,ARAIHAZAR,C16-0166,URBAN WET MARKET GROCER,8.602,8.408,0.194,97.255282,SYLHET
...,...,...,...,...,...,...,...,...,...
573658,7,TEKNAF,C28-2627318,URBAN NEIGHBORHOOD GROCER,0.178,0.178,0.000,100.000000,CTG
573659,7,TEKNAF,C28-2627319,URBAN NEIGHBORHOOD GROCER,1.046,1.046,0.000,100.000000,CTG
573660,7,TEKNAF,C28-2627320,URBAN NEIGHBORHOOD GROCER,0.230,0.230,0.000,100.000000,CTG
573661,7,TEKNAF,C28-2633540,CONSUMER DURABLE OUTLET,10.000,6.000,4.000,60.000000,CTG


In [3]:
# read outlet type
file = r"C:\Users\Shithi.Maitra\Downloads\Apr'23 IHT.xlsb"
outlet_df = pd.read_excel(open(file, "rb"), sheet_name="Sheet2", header=3, index_col=None)
outlet_df = duckdb.query('''select OutletCode outlet, Type outlet_typ from outlet_df''').df()

In [4]:
# join
qry = '''
select *, case when outlet_typ is null then 'unspecified' else outlet_typ end outlet_type
from 
    sccf_df tbl1 
    left join 
    outlet_df tbl2 using(outlet)
'''
sccf_df = duckdb.query(qry).df()
display(sccf_df)

,month,town,outlet,channel,ord_qty,inv_qty,loss_qty,ccf,region,outlet_typ,outlet_type
0,4,BAGERHAT,K24-0852,URBAN NEIGHBORHOOD GROCER,1.792,1.762,0.031,98.284224,KHULNA,Tail,Tail
1,4,BAGERHAT,K24-0853,URBAN NEIGHBORHOOD GROCER,1.906,1.839,0.066,96.511863,KHULNA,Tail,Tail
2,4,BAGERHAT,K24-0855,URBAN NEIGHBORHOOD GROCER,1.982,1.851,0.131,93.411435,KHULNA,Tail,Tail
3,4,BAGERHAT,K24-0859,RURAL NEIGHBORHOOD GROCER,1.649,1.647,0.002,99.859646,KHULNA,Tail,Tail
4,4,BAGERHAT,K24-0861,URBAN NEIGHBORHOOD GROCER,0.042,0.042,0.000,100.000000,KHULNA,Tail,Tail
...,...,...,...,...,...,...,...,...,...,...,...
3211861,7,HATHAZARI,C41-2353859,RURAL NEIGHBORHOOD GROCER,0.862,0.862,0.000,100.000000,CTG,Tail,Tail
3211862,7,HATHAZARI,C41-2410451,RURAL NEIGHBORHOOD GROCER,0.291,0.291,0.000,100.000000,CTG,Tail,Tail
3211863,7,HATHAZARI,C41-2410611,URBAN NEIGHBORHOOD GROCER,1.784,1.784,0.000,100.000000,CTG,Tail,Tail
3211864,7,HATHAZARI,C41-2410650,URBAN NEIGHBORHOOD GROCER,0.153,0.141,0.012,92.236599,CTG,Tail,Tail


In [5]:
# loss + CCF
qry = '''
select month, 'loss' attr, region, town, channel, outlet_type, outlet, sum(loss_qty) val from sccf_df group by 1, 2, 3, 4, 5, 6, 7
union all
select month, 'SCCF' attr, region, town, channel, outlet_type, outlet, sum(inv_qty)*1.00/sum(ord_qty) val from sccf_df group by 1, 2, 3, 4, 5, 6, 7    
'''
ccfloss_df = duckdb.query(qry).df()
display(ccfloss_df)

,month,attr,region,town,channel,outlet_type,outlet,val
0,5,loss,CS,NOWABGANJ,RURAL NEIGHBORHOOD GROCER,Tail,D05-1739,0.000000
1,5,loss,CS,NOWABGANJ,RURAL NEIGHBORHOOD GROCER,Harvest,D05-04535,0.000000
2,5,loss,CS,NOWABGANJ,RURAL NEIGHBORHOOD GROCER,Tail,D05-2331965,0.000000
3,5,loss,CS,NOWABGANJ,SHOPPING COMPEX/ COSMETIC SHOP,unspecified,D05-0210,0.042000
4,5,loss,CS,NOWABGANJ,RURAL NEIGHBORHOOD GROCER,Tail,D05-1785,0.000000
...,...,...,...,...,...,...,...,...
6423727,5,SCCF,BOGRA,JOYPURHAT,TONG,unspecified,D21-04418,0.983213
6423728,5,SCCF,BOGRA,JOYPURHAT,RURAL COSMETIC STORES,unspecified,D21-06126,0.992400
6423729,5,SCCF,BOGRA,JOYPURHAT,RURAL NEIGHBORHOOD GROCER,Tail,D21-1754841,1.000000
6423730,5,SCCF,BOGRA,JOYPURHAT,URBAN NEIGHBORHOOD GROCER,Tail,D21-05596,0.902819


In [6]:
# pivot detailed
piv_df = pd.pivot_table(
    ccfloss_df, 
    values='val', 
    index=['region', 'town', 'channel', 'outlet_type', 'outlet'], 
    columns=['month', 'attr'], 
    aggfunc='sum'
    ).reset_index()
display(piv_df)

month   region      town                         channel  outlet_type  \
attr                                                                    
0        BOGRA    BAAGRA  CONVINIENCE STORES OUT OF HOME  unspecified   
1        BOGRA    BAAGRA                     DRUG STORES  unspecified   
2        BOGRA    BAAGRA                     DRUG STORES  unspecified   
3        BOGRA    BAAGRA                     DRUG STORES  unspecified   
4        BOGRA    BAAGRA                     DRUG STORES  unspecified   
...        ...       ...                             ...          ...   
488083  SYLHET  ZAKIGANJ                       WHOLESALE  unspecified   
488084  SYLHET  ZAKIGANJ                       WHOLESALE  unspecified   
488085  SYLHET  ZAKIGANJ                       WHOLESALE  unspecified   
488086  SYLHET  ZAKIGANJ                       WHOLESALE  unspecified   
488087  SYLHET  ZAKIGANJ                       WHOLESALE  unspecified   

month        outlet         1                2                3         \
attr                     SCCF   loss      SCCF   loss      SCCF   loss   
0       K70-2092223  1.000000  0.000  1.000000  0.000  1.000000  0.000   
1         K28-12578  1.000000  0.000  0.786325  0.625  0.593640  0.115   
2         K28-12579  1.000000  0.000  1.000000  0.000  0.557447  0.104   
3         K28-12580  1.000000  0.000  0.466667  0.008  0.347305  0.109   
4         K28-12581  1.000000  0.000  0.488372  0.022  0.448000  0.069   
...             ...       ...    ...       ...    ...       ...    ...   
488083  C58-2312074  0.976565  0.438  0.993136  0.083  0.996468  0.042   
488084  C58-2312075  1.000000  0.000  1.000000  0.000  1.000000  0.000   
488085  C58-2312076  1.000000  0.000  0.862360  1.813  0.982292  0.333   
488086  C58-2341901  0.853131  1.229  0.951877  1.104  0.959825  0.917   
488087  C58-2359925  1.000000  0.000  1.000000  0.000  1.000000  0.000   

month          4                5                6                7         
attr        SCCF   loss      SCCF   loss      SCCF   loss      SCCF   loss  
0       1.000000  0.000  1.000000  0.000  1.000000  0.000       NaN    NaN  
1       0.817241  0.053  1.000000  0.000  0.988615  0.012       NaN    NaN  
2       0.843985  0.083  1.000000  0.000  0.831169  0.026       NaN    NaN  
3       0.892857  0.028  1.000000  0.000  1.000000  0.000       NaN    NaN  
4       0.895131  0.028  1.000000  0.000  1.000000  0.000       NaN    NaN  
...          ...    ...       ...    ...       ...    ...       ...    ...  
488083  0.769514  0.563  0.903697  0.917  0.941273  0.500  0.961054  0.417  
488084  1.000000  0.000  0.824945  6.000  0.972820  0.520  1.000000  0.000  
488085  0.968980  0.240  0.985122  0.167  0.976712  0.333  1.000000  0.000  
488086  0.915852  1.050  0.990668  0.214  0.968551  0.804  0.995285  0.083  
488087  0.905117  1.260  1.000000  0.000  1.000000  0.000  0.978589  0.125  

[488088 rows x 19 columns]

In [7]:
# pivot region
qry = '''
select month, 'loss' attr, region, sum(loss_qty) val from sccf_df group by 1, 2, 3
union all
select month, 'SCCF' attr, region, sum(inv_qty)*1.00/sum(ord_qty) val from sccf_df group by 1, 2, 3
'''
region_df = duckdb.query(qry).df()
region_df = pd.pivot_table(region_df, values='val', index=['region'], columns=['month', 'attr'], aggfunc='sum').reset_index()

# pivot town
qry = '''
select month, 'loss' attr, town, sum(loss_qty) val from sccf_df group by 1, 2, 3
union all
select month, 'SCCF' attr, town, sum(inv_qty)*1.00/sum(ord_qty) val from sccf_df group by 1, 2, 3
'''
town_df = duckdb.query(qry).df()
town_df = pd.pivot_table(town_df, values='val', index=['town'], columns=['month', 'attr'], aggfunc='sum').reset_index()

# pivot channel
qry = '''
select month, 'loss' attr, channel, sum(loss_qty) val from sccf_df group by 1, 2, 3
union all
select month, 'SCCF' attr, channel, sum(inv_qty)*1.00/sum(ord_qty) val from sccf_df group by 1, 2, 3
'''
channel_df = duckdb.query(qry).df()
channel_df = pd.pivot_table(channel_df, values='val', index=['channel'], columns=['month', 'attr'], aggfunc='sum').reset_index()

# pivot outlet_type
qry = '''
select month, 'loss' attr, outlet_type, sum(loss_qty) val from sccf_df group by 1, 2, 3
union all
select month, 'SCCF' attr, outlet_type, sum(inv_qty)*1.00/sum(ord_qty) val from sccf_df group by 1, 2, 3
'''
outlet_type_df = duckdb.query(qry).df()
outlet_type_df = pd.pivot_table(outlet_type_df, values='val', index=['outlet_type'], columns=['month', 'attr'], aggfunc='sum').reset_index()

# pivot outlet
qry = '''
select month, 'loss' attr, outlet, sum(loss_qty) val from sccf_df group by 1, 2, 3
union all
select month, 'SCCF' attr, outlet, sum(inv_qty)*1.00/sum(ord_qty) val from sccf_df group by 1, 2, 3
'''
outlet_df = duckdb.query(qry).df()
outlet_df = pd.pivot_table(outlet_df, values='val', index=['outlet'], columns=['month', 'attr'], aggfunc='sum').reset_index()

In [8]:
# store
with pd.ExcelWriter("C:/Users/Shithi.Maitra/Downloads/loss_vs_CCF.xlsx") as writer:
    piv_df.to_excel(writer, sheet_name="Full")
    region_df.to_excel(writer, sheet_name="Region")
    town_df.to_excel(writer, sheet_name="Town")
    channel_df.to_excel(writer, sheet_name="Channel")
    outlet_type_df.to_excel(writer, sheet_name="Type")
    outlet_df.to_excel(writer, sheet_name="Outlet")

In [9]:
# # sanity
# qry = '''
# select outlet, count(*) instances
# from outlet_df
# group by 1
# order by 2 desc
# '''
# df = duckdb.query(qry).df()
# display(df)